# Imports and magics

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
from nltk.tokenize import word_tokenize
import pickle
from sklearn.cluster import AgglomerativeClustering
from collections import Counter, defaultdict
from sklearn.neighbors import DistanceMetric
from math import gcd
from functools import reduce
import seaborn as sns
from scipy.stats import spearmanr

In [2]:
%matplotlib inline

from IPython.core.display import display, HTML
display(HTML("<style>.container {width:90% !important;}</style>"))

# Parameters

In [3]:
letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

cipher_params = {'min_length': 500,
                 'max_length': 1000,
                 'messages_per_source': 10,
                 'key_length': 5}

decipher_params = {'n_top_trigrams_per_message': 100,
                   'n_grams': 3,
                   'kasiski_n': 4,
                   'min_repeats': 6,
                   'max_period': 100,
                   'monte_carlo_samples_key_len': 10,
                   'monte_carlo_num_trials': 1000,
                   'monte_carlo_sample_fraction': 6}

vector_params = {'top_N': 10,
                 'min_matches': 3}

agg_params = {'affinity': 'precomputed',
              'linkage': 'single'}

# Read in and clean raw text
### Read in all the source documents
### Replace digits, newlines, and other escapes with spaces
### Uppercase

In [4]:
def stringify_document(filepath):
    with open(filepath, 'r') as f:
        doc_as_string = f.read()
        f.close()
        return doc_as_string

def clean_string(string):
    return re.sub('[^a-zA-Z]', ' ', string).upper()

def clean_docs(folderpath):
    doc_strings = [stringify_document(os.path.join(folderpath, name)) for name in os.listdir(folderpath)]
    clean_doc_strings = [clean_string(string) for string in doc_strings]
    return clean_doc_strings

# Create ciphertexts
### Tokenize plaintext
###  Extract 10 plaintexts from each source, of variable length, for each document
### Encipher plaintexts

In [5]:
def get_plaintext_fragment(word_tokens):
    length = np.random.randint(cipher_params['min_length'], cipher_params['max_length'])
    start_point = np.random.randint(0, len(word_tokens) - length)
    cand_plaintext = ''.join(word_tokens[start_point: start_point + length])
    while len(cand_plaintext) % cipher_params['key_length'] != 0:
        cand_plaintext = ''.join([cand_plaintext, 'A'])
    return cand_plaintext


def encrypt_message(key, message):
    return translate_message(key, message, 'encrypt')

def decrypt_message(key, message):
    return translate_message(key, message, 'decrypt')

def translate_message(key, message, mode):
    
    key_indices = [letters.index(key_letter) for key_letter in key]
    
    def cipher_letter(letter, key_index):
        if mode == 'encrypt':
            offset = key_index
        elif mode == 'decrypt':
            offset = -key_index
        
        return letters[(letters.index(letter) + offset) % len(letters)]
    
    return ''.join([cipher_letter(message[i], key_indices[i % len(key)]) for i in range(len(message))])

In [6]:
def clean_and_token_document(filepath):
    print('filepath', filepath)
    return word_tokenize(clean_string(stringify_document(filepath)))

def divide_doc_into_messages(words, key):
    return [encrypt_message(key, get_plaintext_fragment(words))
            for _ in range(cipher_params['messages_per_source'])]

def generate_key(length):
    return ''.join([letters[np.random.randint(26)] for _ in range(length)])


def gen_ciphered_texts(source_folder):
    num_docs = len(os.listdir(source_folder))
    ciphertexts = {}
    
    ciphertexts['document_keys'] = [generate_key(cipher_params['key_length'])
                                    for _ in range(num_docs)]
    
    doc_filepaths = [os.path.join(source_folder, name) for name in os.listdir(source_folder)]
    
    ciphertexts['ciphertexts'] = [divide_doc_into_messages(clean_and_token_document(doc_filepaths[i]),
                                                           ciphertexts['document_keys'][i])
                                  for i in range(num_docs)]
    
    return ciphertexts

# Tokenize ciphertexts
### Shuffle, then tag ciphertexts with unique IDs

In [7]:
def shuffle_texts(texts):
    texts = np.array(texts).ravel()
    np.random.shuffle(texts)
    return texts

def tag_shuffled_texts(shuffled_texts):
    return {i: {'text': text} for i, text in enumerate(shuffled_texts)}

def shuffle_and_tag(texts):
    return tag_shuffled_texts(shuffle_texts(texts))

In [8]:
def generate_ciphertexts(folderpath):
    document_ciphertexts = gen_ciphered_texts(folderpath)
    message_ciphertexts = shuffle_and_tag(document_ciphertexts['ciphertexts'])
    return message_ciphertexts

In [9]:
tng_messages = generate_ciphertexts('data/tng_val')

filepath data/tng_val/294-0.txt
filepath data/tng_val/108-0.txt
filepath data/tng_val/2350-0.txt
filepath data/tng_val/pg139.txt
filepath data/tng_val/3289-0.txt
filepath data/tng_val/pg126.txt
filepath data/tng_val/2852-0.txt
filepath data/tng_val/pg537.txt
filepath data/tng_val/290-0.txt
filepath data/tng_val/903-0.txt
filepath data/tng_val/pg2097.txt
filepath data/tng_val/356-0.txt
filepath data/tng_val/48320-0.txt
filepath data/tng_val/834-0.txt
filepath data/tng_val/pg2344.txt


# Extract features for clustering
### Split ciphertexts into tokens
### Get vectorized counts over whole corpus of ciphertexts, top 10 per message

In [10]:
def populate_ngrams(texts, n):
    for text_index in texts:
        texts[text_index]['%s_grams' % str(n)] = [texts[text_index]['text'][k: k + n]
                                                   for k in range(len(texts[text_index]['text']) - n)]
        
def top_N_in_doc(ngrams, top_N=vector_params['top_N']):
    ngram_counts = Counter(ngrams).most_common(top_N)
    return list([ngram for ngram in zip(*ngram_counts)][0])

def populate_most_common_ngrams(texts, n, top_N=vector_params['top_N']):
    for text_index in texts:
        texts[text_index]['common_%s_grams' % str(n)] = top_N_in_doc(texts[text_index]['%s_grams' % str(n)])
        
def populate_n_grams_list(texts, n):
    populate_ngrams(texts, n)
    populate_most_common_ngrams(texts, n)

# Cluster messages by similarity
## Group together any texts with at least 3 ngrams in common
## Cluster ciphertexts

In [11]:
def num_ngrams_in_common(ngram1, ngram2):
    return len(set(ngram1).intersection(set(ngram2)))

def populate_matching_messages(texts, ngrams='common_3_grams', min_matches=vector_params['min_matches']):
    num_texts = len(texts.keys())
    for i in range(num_texts):
        messages_with_n_common = []
        for j in range(num_texts):
            num_in_common = num_ngrams_in_common(texts[i][ngrams], texts[j][ngrams])
            if num_in_common > 0:
                messages_with_n_common.append(j)
        texts[i]['matched_messages'] = messages_with_n_common

In [12]:
def make_array_of_matches(texts):
    num_messages = len(texts.keys())
    matches = np.zeros((num_messages, num_messages))
    
    for i in range(num_messages):
        matches[i][i] = 1
        for j in texts[i]['matched_messages']:
            matches[i][j] = 1
            
    return matches


def assign_clusters(matches, texts):
    match_metric = DistanceMetric.get_metric('dice')
    match_dist_mat = match_metric.pairwise(matches)
    
    clusters = AgglomerativeClustering(n_clusters=int(len(texts) / cipher_params['messages_per_source'])
                                       , affinity=agg_params['affinity']
                                       , linkage=agg_params['linkage'])
    clusters.fit(match_dist_mat)
    
    for i in range(len(texts)):
        texts[i]['assigned_cluster'] = clusters.labels_[i]
        
        
def get_cluster_assignments(texts, n_grams=decipher_params['n_grams']):
    populate_n_grams_list(texts, n_grams)
    populate_matching_messages(texts)
    assign_clusters(make_array_of_matches(texts), texts)
    
    
def combine_clusters(texts):
    num_clusters = max([texts[i]['assigned_cluster'] for i in range(len(texts))]) + 1
    clustered_texts = {i: {'text': ''} for i in range(num_clusters)}
    
    for i in range(len(texts)):
        cluster_num = texts[i]['assigned_cluster']
        clustered_texts[cluster_num]['text'] = ''.join([clustered_texts[cluster_num]['text'], texts[i]['text']])
        
    return clustered_texts

In [13]:
def cluster_ciphertexts(messages):
    get_cluster_assignments(messages, n_grams=decipher_params['n_grams'])
    return combine_clusters(messages)

In [14]:
tng_ciphers = cluster_ciphertexts(tng_messages)

In [15]:
tng_ciphers[0]['text'][:1000]

'NBHJMFFPFZONHIKZCQXXFSRKFBYWLBWIUOUXIUTTCWKOMHIXQECXHQTLYHRZNYQXZNYGYQXLRLYMBHXSUCQTMMMHFLYXZFFBUVMMMGRCYYLUFYYHWQTYLHXDYLHXXFSVLYYPHOKOHLNGYZHXFOLHPMVIXQFBCVZMMYZXFMIQPMCXKBUNBLKWCNLPFCGHKAQNKXFQYWLAEIXORLCHKPFYVQDUXHFZNIRRDWIQCUXYQZQBYKXEBUGEUMFLQFFYVJUFYDQAOLHUBYHVBMHXSBDBUSPIYGDVPIUVJGWBEVTCGLCYSLHXPCHJLRNBLPBLIEIQGJULHYMWLNYWRODYWWVQMSHPUNBLKWCMHBTIQZBEBIXIPUJSOAUWKFFNBHPOINOXZXSDOPCHVMQWNROIUMVQUFFZOUNCQDUHNKBBULOLGLQKBZBIOJQMCQQQLLXMFYXKFYCOQAQLMWLAXNKXFSIXTQLYZOUNCQDMLYSLDNIIQTCMFXEYMDFPBYVLUUGGLZNSRRFBCQHUNGDVNYUOFFNFHMDYGDQGLYLZMHNKBXJNKFZECQDFBUWVAOLHSUXYQZQCMQLFWIPMXYNHIQMNUXPYEQBIGSIOUYHGQAIQHIXNIGFELYJXDXBLPIILGPTYFDFPXIZKTCMSBZUHGIAIEHAOOLLLGMFBXFBCPTTUNGLKIOPBMHGUEAFGHPAHFBQTUNWEQLYLPMHCPMALNDKFQCWKQMMZEAGSRRTUPHKANMHBZWUQVAOJULPOWHEUGCWEUHELZMHNKBZXIVLUQCOIPIGBYQMNKLIGUQVOIHVQMVFHPTUPHVAONKBDYUUBFBLHBICNKFZWUOIQRWHIXYHWPMCXKLXGYVJMSCDPWCZWEQSUUBMFFOXDAYDYXYVRAUYXPBZQCWEBIQHOROFYLUWYVFTUPHKAXIXYFNBHVMLYWEAOAKFRUCOQAMYHTTUNWEQCLYLUWYVEMPYWLPIQLQTCNSBDBUSPUWUQEQFJBLGNIVB

# Break ciphertexts

## Kasiski index

### Find all repeated 4-grams, and their positions
### Find most common gcd among all repeated 4-gram periods

In [16]:
def gen_ngram_positions(texts, n=decipher_params['kasiski_n']):
    for text_index in texts:
        all_ngrams = {}
        
        for k in range(len(texts[text_index]['text']) - n):
            ngram = texts[text_index]['text'][k: k + n]
            if ngram not in all_ngrams.keys():
                all_ngrams[ngram] = []
            all_ngrams[ngram].append(k)
        
        texts[text_index]['%s_gram_positions' % str(n)] = {ngram: all_ngrams[ngram]
                                                           for ngram in all_ngrams
                                                           if len(all_ngrams[ngram]) >= decipher_params['min_repeats']}

        
def gen_ngram_periods(texts, n=decipher_params['kasiski_n']):
    
    period_key = '%s_gram_periods' % str(n)
    position_key = '%s_gram_positions' % str(n)
    
    def get_periods(positions):
        # print(ngram, len(positions), positions)
        return [positions[i] - positions[i - 1]
                for i in range(len(positions) - 1, 0, -1)
                if positions[i] - positions[i - 1] < decipher_params['max_period']]
    
    for text_index in texts:
        texts[text_index][period_key] = [get_periods(texts[text_index][position_key][ngram])
                                            for ngram in texts[text_index][position_key]
                                            if len(get_periods(texts[text_index][position_key][ngram])) > 0]
        
        texts[text_index][period_key] = set([item 
                                             for periods in texts[text_index][period_key]
                                             for item in periods])
        

def gen_key_length(texts, n=decipher_params['kasiski_n']):

    def gcd_list(nums):
        return reduce(gcd, nums)
    
    def monte_carlo_key_len(periods):
        # print(periods)
        gcds = []
        samples_per_trial = int(len(periods) / decipher_params['monte_carlo_sample_fraction'])
        
        for i in range(decipher_params['monte_carlo_num_trials']):
            cand_gcd = gcd_list(np.random.choice(list(periods), samples_per_trial, replace=False))
            if cand_gcd > 1:
                gcds.append(cand_gcd)
        # print(gcds)        
        return Counter(gcds).most_common(1)[0][0]
    
    for text_index in texts:
        try:
            texts[text_index]['key_length'] = monte_carlo_key_len(texts[text_index]['%s_gram_periods' % str(n)])
        except TypeError:
            texts[text_index]['key_length'] = texts[text_index - 1]['key_length']

In [17]:
def est_kasiski_index(texts):
    gen_ngram_positions(texts)
    gen_ngram_periods(texts)
    gen_key_length(texts)

# Get frequency counts for each alphabet in key

In [18]:
def frequency_counts(ciphertext, key_len):
    return [Counter(ciphertext[i::key_len]) for i in range(key_len)]

def gen_freq_counts(texts):
    for text_index in texts:
        texts[text_index]['freq_counts'] = frequency_counts(texts[text_index]['text'], texts[text_index]['key_length'])

In [19]:
def gen_K_index_and_counts(texts):
    est_kasiski_index(texts)
    gen_freq_counts(texts)

# Guess that the key is the one that makes E the most common letter in the plaintext

In [20]:
def rev_eng_key_letter(cand_key_letter, guess):
    return letters[(letters.index(cand_key_letter) - letters.index(guess)) % 26]

def rev_eng_key(cand_key):
    guess = ''.join(['E' for _ in range(len(cand_key))])
    return ''.join([rev_eng_key_letter(cand_key[i], guess[i]) for i in range(len(cand_key))])

In [21]:
def gen_keys(texts):
    for text_index in texts:
        texts[text_index]['key'] = rev_eng_key(''.join([texts[text_index]['freq_counts'][i].most_common(1)[0][0] for i in range(texts[text_index]['key_length'])]))

def gen_plaintexts(texts):
    for text_index in texts:
        texts[text_index]['plaintext'] = translate_message(texts[text_index]['key'], texts[text_index]['text'], 'decrypt')

In [22]:
def decrypt_all_ciphertexts(texts):
    gen_K_index_and_counts(texts)
    gen_keys(texts)
    gen_plaintexts(texts)

In [23]:
decrypt_all_ciphertexts(tng_ciphers)
tng_ciphers[12]['plaintext'][:1000]

'TOOFORALLTHATHEWASSOSHORTMAHOMETSINGHWASLEFTTOGUARDTHEDOORWETOOKHIMTOAPLACEWHICHTHESIKHSHADALREADYPREPAREDITWASSOMEDISTANCEOFFWHEREAWINDINGPASSAGELEADSTOAGREATEMPTYHALLTHEBRICKWALLSOFWHICHWEREALLCRUMBLINGTOPIECESTHEEARTHFLOORHADSUNKINATONEPLACEMAKINGANATURALGRAVESOWELEFTACHMETTHEMERCHANTTHEREHAVINGFIRSTCOVEREDHIMOVERWITHLOOSEBRICKSTHISDONEWEALLWENTBACKTOTHETREASUREITLAYWHEREHEHADDROPPEDITWHENHEWASFIRSTATTACKEDTHEBOXWASTHESAMEWHICHNOWLIESOPENUPONYOURTABLEAKEYWASHUNGBYASILKENCORDTOTHATCARVEDHANDLEUPONTHETOPWEOPENEDITANDTHELIGHTOFTHELANTERNGLEAMEDUPONACOLLECTIONOFGEMSSUCHASIHAVEREADOFANDTHOUGHTABOUTWHENIWASALITTLELADATPERSHOREITWASBLINDINGTOLOOKUPONTHEMWHENWEHADFEASTEDOUREYESWETOOKTHEMALLOUTANDMADEALISTOFTHEMTHEREWEREONEHUNDREDANDFORTYTHREEDIAMONDSOFTHEFIRSTWATERINCLUDINGONEWHICHHASBEENCALLEDIBELIEVETHEGREATMOGULANDISSAIDTOBETHESECONDLARGESTSTONEINEXISTENCETHENTHEREWERENINETYSEVENVERYFINEEMERALDSANDONEHUNDREDANDSEVENTYRUBIESSOMEOFWHICHHOWEVERWERESMALLTHEREWEREFORTYCARBUNCLESTWOHUNDREDAND

# Put it all together and test

In [24]:
def split_shuffle_encrypt_cluster_decrypt(folderpath):
    messages = generate_ciphertexts(folderpath)
    ciphertexts = cluster_ciphertexts(messages)
    decrypt_all_ciphertexts(ciphertexts)
    return ciphertexts

In [25]:
test_texts = split_shuffle_encrypt_cluster_decrypt('data/test')

for text in test_texts:
    print(test_texts[text]['plaintext'][:100])

filepath data/test/pg456.txt
filepath data/test/pg159.txt
filepath data/test/pg35461.txt
filepath data/test/pg1013.txt
filepath data/test/pg5230.txt
ROLLEDTHEINSECTINHIMWASCONFESSEDITSIMPLYILLUSTRATESTHEUNTHINKINGWAYINWHICHONEACQUIRESHABITSOFFEELING
CROWNOFHISHEADLOOKOUTSAIDEVERYBODYFENCINGATRANDOMANDHITTINGATNOTHINGHOLDHIMSHUTTHEDOORDONTLETHIMLOOS
WHICHHEHADLEFTMETHEBLACKWINDOWSTAREDATMELIKEANEYEATLASTWITHANEFFORTIPUTOUTTHELIGHTANDGOTINTOTHEHAMMO
MIGHTSPEAKTOMEFORALITTLETIMEAPARTNOISAIDIHAVENOSECRETSFROMTHISLADYWHATDOYOUWANTTOTELLMEHESAIDITWASAT
LANGUAGESRHINELANDRHINOCEROSRHODESRHODESIARHODESIANMANRICHELIEUCARDINALRICHMONDUSAROADSROBERTSONROBE


# Determine if plaintext has been translated into English

In [26]:
overall_monogram_freqs = pd.read_csv('data/english_monograms.txt', sep=' ', names=['count'])
overall_monogram_freqs['freqs'] = overall_monogram_freqs['count'] / overall_monogram_freqs['count'].sum()
overall_monogram_freqs.drop('count', axis=1, inplace=True)

monogram_order = list(overall_monogram_freqs.index)[:12]
print(monogram_order)

['E', 'T', 'A', 'O', 'I', 'N', 'S', 'R', 'H', 'L', 'D', 'C']


In [27]:
def gen_plaintext_letter_order(texts):
    for text_index in texts:
        ordered_letters = Counter(texts[text_index]['plaintext']).most_common(26)
        texts[text_index]['plain_freq_order'] = [ordered_letters[i][0] for i in range(12)]
        
def is_plaintext(text):
    corr, _ = spearmanr(monogram_order, text['plain_freq_order'])
    if corr > 0:
        return True
    else:
        return False

In [28]:
gen_plaintext_letter_order(tng_ciphers)

In [29]:
print(tng_ciphers[0]['plain_freq_order'])

['E', 'T', 'O', 'A', 'I', 'N', 'S', 'H', 'R', 'L', 'D', 'U']


# Calculate recall over the Gutenberg dataset

In [30]:
gutenberg = split_shuffle_encrypt_cluster_decrypt('data/metrics')

filepath data/metrics/Robert Louis Stevenson___Prince Otto.txt
filepath data/metrics/Edgar Rice Burroughs___Tarzan and the Jewels of Opar.txt
filepath data/metrics/Anthony Trollope___Cousin Henry.txt
filepath data/metrics/Jerome Klapka Jerome___They and I.txt
filepath data/metrics/Anthony Trollope___The Fixed Period.txt
filepath data/metrics/Ambrose Bierce___The Devil's Dictionary.txt
filepath data/metrics/P G Wodehouse___Uneasy Money.txt
filepath data/metrics/Hamlin Garland___Wayside Courtships.txt
filepath data/metrics/P G Wodehouse___The Prince and Betty.txt
filepath data/metrics/Edward Phillips Oppenheim___The Double Four.txt
filepath data/metrics/Charles Kingsley___Twenty-Five Village Sermons.txt
filepath data/metrics/William Makepeace Thackeray___Men's Wives.txt
filepath data/metrics/Grant Allen___An African Millionaire.txt
filepath data/metrics/Baronness Orczy___Castles in the Air.txt
filepath data/metrics/Nathaniel Hawthorne___Passages from the American Notebooks, Volume 2.txt


filepath data/metrics/Hamlin Garland___Victor Ollnee's Discipline.txt
filepath data/metrics/Frank Richard Stockton___A Bicycle of Cathay.txt
filepath data/metrics/Edward Phillips Oppenheim___The Survivor.txt
filepath data/metrics/Anthony Trollope___The Struggles of Brown, Jones, and Robinson.txt
filepath data/metrics/William Henry Hudson___Birds and Man.txt
filepath data/metrics/James Otis___A Runaway Brig.txt
filepath data/metrics/George Alfred Henty___Among Malay Pirates.txt
filepath data/metrics/G K Chesterton___Heretics.txt
filepath data/metrics/Anthony Trollope___The Golden Lion of Granpere.txt
filepath data/metrics/Edward Phillips Oppenheim___The Governors.txt
filepath data/metrics/Frank Richard Stockton___The Associate Hermits.txt
filepath data/metrics/Edward Stratemeyer___The Rover Boys in Camp.txt
filepath data/metrics/Baronness Orczy___I Will Repay.txt
filepath data/metrics/William Dean Howells___Through the Eye of the Needle.txt
filepath data/metrics/Anthony Trollope___Sir H

In [31]:
gen_plaintext_letter_order(gutenberg)

english_plaintext_count = 0
for text in gutenberg:
    if is_plaintext(gutenberg[text]):
        english_plaintext_count += 1
        
print(english_plaintext_count)
print(english_plaintext_count/ len(gutenberg))

113
0.7583892617449665


/home/bam/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:248: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)
